In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.sparse import csr_matrix
from scipy.stats import mannwhitneyu

# *NOTES
Methylation data has been pre-processed and standardized (Esteban-Cantos et al., 2023)

# 1. Hierarchical clustering of methylation levels in HIV 

## Infection
HIV- vs HIV+/pre-ART

In [ ]:
# Load DNA methylation data as a pandas DataFrame
methylation_data = pd.read_csv('Data/methylation_values.csv', index_col=0)
methylation_data_infection = methylation_data.drop('β-value_HIV+ post-ART', axis=1)
cpg_ids = methylation_data_infection.pop('CpG_ID')

# Drop CpG_IDs with NaN (NOTE: there should be none)
methylation_data_infection.dropna(inplace=True)

# Saving to csv
methylation_data_infection.to_csv('Data/methylation_data_infection.csv')
methylation_data_infection

In [ ]:
# Generate a clustermap 
sns.clustermap(methylation_data_infection, cmap='coolwarm')

# Display the heatmap
plt.show()

In [ ]:
# Choosing the optimal number of clusters using k-means and calculating within-cluster sum of squares for each number 
# of clusters
inertias = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(methylation_data_infection)
    inertias.append(kmeans.inertia_)

# Plot the number of clusters against the within-cluster sum of squares
plt.plot(range(1, 11), inertias, 'bx-')
plt.xlabel('Number of Clusters')
plt.ylabel('Within-Cluster Sum of Squares')
plt.title('Elbow Method: Infection')
plt.show()

In [ ]:
# Silhouette coefficient using optimal number of clusters
kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=100)
kmeans.fit(methylation_data_infection)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

silhouette_avg = silhouette_score(methylation_data_infection, labels)
print("The average silhouette coefficient is :", silhouette_avg)

In [ ]:
# Clustering by methylation level
# Calculating linkage matrix
linkages = linkage(methylation_data_infection, method='average') # Computing linkage matrix using average method

k = 2 # Number of clusters
clusters = fcluster(linkages, k, criterion='maxclust') # Assigns observations to at most k clusters
colors = sns.color_palette("Set1", n_colors=k)
row_colors = [colors[label - 1] for label in clusters] # Red (cluster 1) is hyper, blue (cluster 2) is hypo

sns.clustermap(methylation_data_infection, row_colors=row_colors, cmap='coolwarm') 

silhouette_avg = silhouette_score(methylation_data_infection, clusters)
print("The average silhouette coefficient is :", silhouette_avg)

In [ ]:
# Extracting CpG_IDs for each methylation level cluster
cpg_ids = np.array(cpg_ids)
infection_clusters = np.array(clusters)
hyper = []
hypo = []

for i, cluster in enumerate(infection_clusters):
    if cluster == 1:
        hyper.append(cpg_ids[i])
    else:
        hypo.append(cpg_ids[i])
        
# Saving to csv files 
infection_cpgs_high = pd.DataFrame(hyper, columns=['CpG_ID'])
infection_cpgs_high.to_csv('Data/infection_cpgs_high.csv')
infection_cpgs_low = pd.DataFrame(hypo, columns=['CpG_ID'])
infection_cpgs_low.to_csv('Data/infection_cpgs_low.csv')

## Treatment
HIV+/pre-ART vs HIV+/post-ART

In [ ]:
# Load DNA methylation data as a pandas DataFrame
methylation_data = pd.read_csv('Data/treatment_df.csv', index_col=0)
methylation_data_treatment = methylation_data.drop('Δβ-value_HIV+ pre-ART vs. HIV+ post-ART', axis=1)
cpg_ids = methylation_data_treatment.pop('CpG_ID')

# Drop CpG_IDs with NaN (NOTE: there should be none)
methylation_data_treatment.dropna(inplace=True)

# Saving to csv
methylation_data_treatment.to_csv('Data/methylation_data_treatment.csv')
methylation_data_treatment

In [ ]:
# Generate a clustermap 
sns.clustermap(methylation_data_treatment, cmap='coolwarm')

# Display the heatmap
plt.show()

In [ ]:
# Choosing the optimal number of clusters using k-means and calculating within-cluster sum of squares for each number 
# of clusters
inertias = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(methylation_data_treatment)
    inertias.append(kmeans.inertia_)

# Plot the number of clusters against the within-cluster sum of squares
plt.plot(range(1, 11), inertias, 'bx-')
plt.xlabel('Number of Clusters')
plt.ylabel('Within-Cluster Sum of Squares')
plt.title('Elbow Method: Treatment')
plt.show()

In [ ]:
# Silhouette coefficient using optimal number of clusters
kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=100)
kmeans.fit(methylation_data_treatment)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

silhouette_avg = silhouette_score(methylation_data_treatment, labels)
print("The average silhouette coefficient is :", silhouette_avg)

In [ ]:
# Clustering by methylation level
# Calculating linkage matrix
linkages = linkage(methylation_data_treatment, method='average') # Computing linkage matrix using average method

k = 2 # Number of clusters
clusters = fcluster(linkages, k, criterion='maxclust') # Assigns observations to at most k clusters
colors = sns.color_palette("Set1", n_colors=k)
row_colors = [colors[label - 1] for label in clusters] # So red (cluster 1) is hypo, blue (cluster 2) is hyper

sns.clustermap(methylation_data_treatment, row_colors=row_colors, cmap='coolwarm') 

silhouette_avg = silhouette_score(methylation_data_treatment, clusters)
print("The average silhouette coefficient is :", silhouette_avg)

In [ ]:
# Extracting CpG_IDs for each methylation level cluster
cpg_ids = np.array(cpg_ids)
treatment_clusters = np.array(clusters)
hyper = []
hypo = []

for i, cluster in enumerate(treatment_clusters):
    if cluster == 1:
        hypo.append(cpg_ids[i])
    else:
        hyper.append(cpg_ids[i])
        
# Saving to csv files 
treatment_cpgs_high = pd.DataFrame(hyper, columns=['CpG_ID'])
treatment_cpgs_high.to_csv('Data/treatment_cpgs_high.csv')
treatment_cpgs_low = pd.DataFrame(hypo, columns=['CpG_ID'])
treatment_cpgs_low.to_csv('Data/treatment_cpgs_low.csv')

# 2. Violin plots depicting mean methylation levels

## Infection

In [ ]:
ax = sns.violinplot(data=methylation_data_infection, kde=True)
ax.set_ylim(0, 1)
plt.ylabel('Mean methylation levels')
plt.xlabel('Infection')

## Treatment

In [ ]:
ax = sns.violinplot(data=methylation_data_treatment, kde=True)
ax.set_ylim(0, 1)
plt.ylabel('Mean methylation levels')
plt.xlabel('Treatment')

## Restoration

In [ ]:
ax = sns.violinplot(data=methylation_data_restoration, kde=True)
ax.set_ylim(0, 1)
plt.ylabel('Mean methylation levels')
plt.xlabel('Restoration')

# 3. Methylation levels of CpG clusters

Goal: Trying to characterize the two identified CpG clusters. Are they high vs low methylation level groups? Are they large vs small delta methylation level groups post infection/treatment?

## Infection

In [ ]:
## Complete infection dataframe
# Change in methylation levels after infection
methylation_data_infection['Δβ-value_HIV+ pre-ART vs. HIV-'] = methylation_data_infection['β-value_HIV+ pre-ART'] 
                                                                - methylation_data_infection['β-value_HIV-']
# Mean methylation levels
methylation_data_infection['Mean β-value'] = methylation_data_infection[['β-value_HIV+ pre-ART',
                                                                         'β-value_HIV-']].mean(axis=1)
# CpG cluster associated with each CpG site
methylation_data_infection['CpG_cluster'] = infection_clusters
cluster_dict = {1: 'high', 2: 'low'}
methylation_data_infection['CpG_cluster'] = methylation_data_infection['CpG_cluster'].map(cluster_dict)

methylation_data_infection

In [ ]:
# Separating into two dataframes, one for each DMP cluster
methylation_data_infection_high = methylation_data_infection[methylation_data_infection['CpG_cluster'] == 'high']
methylation_data_infection_low = methylation_data_infection[methylation_data_infection['CpG_cluster'] == 'low']

# Saving to csv
methylation_data_infection_high.to_csv('Data/methylation_data_infection_high.csv')
methylation_data_infection_low.to_csv('Data/methylation_data_infection_low.csv')

In [ ]:
# Violin plot for each of the 2 CpG clusters using the mean β-value
ax = sns.violinplot(data=methylation_data_infection, x='CpG_cluster', y='Mean β-value') 
ax.set_ylim(0, 1)
plt.ylabel('Mean methylation levels')
plt.xlabel('Infection')

In [ ]:
# Mann-Whitney U test
# Separate the data into two groups by CpG cluster
low = methylation_data_infection[methylation_data_infection['CpG_cluster'] == 'low']['Mean β-value']
high = methylation_data_infection[methylation_data_infection['CpG_cluster'] == 'high']['Mean β-value']

statistic, p_value = mannwhitneyu(low, high)
print('Mann-Whitney U statistic:', statistic)
print('p-value:', p_value)

In [ ]:
# Violin plot for each of the 2 CpG clusters using the Δβ-value
ax = sns.violinplot(data=methylation_data_infection, x='CpG_cluster', y='Δβ-value_HIV+ pre-ART vs. HIV-') 
plt.ylabel('Delta methylation levels')
plt.xlabel('Infection')

Thus, CpG clusters can broadly be categorized as group (1) high average methylation level and increase in mean methylation vs (2) low methylation level and decrease in mean methylation.

## Treatment

In [ ]:
## Complete infection dataframe
# Change in methylation levels after infection
methylation_data_treatment['Δβ-value_HIV+ pre-ART vs. HIV+ post-ART'] 
= methylation_data_treatment['β-value_HIV+ pre-ART'] - methylation_data_treatment['β-value_HIV+ post-ART']
# Mean methylation levels
methylation_data_treatment['Mean β-value'] = methylation_data_treatment[['β-value_HIV+ pre-ART',
                                                                         'β-value_HIV+ post-ART']].mean(axis=1)
# CpG cluster associated with each CpG site
methylation_data_treatment['CpG_cluster'] = treatment_clusters
cluster_dict = {1: 'low', 2: 'high'}
methylation_data_treatment['CpG_cluster'] = methylation_data_treatment['CpG_cluster'].map(cluster_dict)

methylation_data_treatment

In [ ]:
# Separating into two dataframes each for DMP cluster
methylation_data_treatment_high = methylation_data_treatment[methylation_data_treatment['CpG_cluster'] == 'high']
methylation_data_treatment_low = methylation_data_treatment[methylation_data_treatment['CpG_cluster'] == 'low']

methylation_data_treatment_high.to_csv('Data/methylation_data_treatment_high.csv')
methylation_data_treatment_low.to_csv('Data/methylation_data_treatment_low.csv')

In [ ]:
# Violin plot for each of the 2 CpG clusters 
ax = sns.violinplot(data=methylation_data_treatment, x='CpG_cluster', y='Mean β-value') # Well what is y?
ax.set_ylim(0, 1)
plt.ylabel('Mean methylation levels')
plt.xlabel('Treatment')

In [ ]:
# Mann-Whitney U test
# Separate the data into two groups
low = methylation_data_treatment[methylation_data_treatment['CpG_cluster'] == 'low']['Mean β-value']
high = methylation_data_treatment[methylation_data_treatment['CpG_cluster'] == 'high']['Mean β-value']

statistic, p_value = mannwhitneyu(low, high)
print('Mann-Whitney U statistic:', statistic)
print('p-value:', p_value)

In [ ]:
# Violin plot for each of the 2 CpG clusters using the Δβ-value
ax = sns.violinplot(data=methylation_data_treatment, x='CpG_cluster', y='Δβ-value_HIV+ pre-ART vs. HIV+ post-ART') 
plt.ylabel('Delta methylation levels')
plt.xlabel('Treatment')

Thus, CpG clusters can broadly be categorized as group (1) high average methylation level and decrease in mean methylation vs (2) low methylation level and increase in mean methylation.